# HW3

http://www.klintonbicknell.com/ling400fall2017/hw/hw3.html

#before calling server in python, need to start server in terminal within the directory where Stanford Corenlp sorce folder is located at
#source folder: C:\Users\Dingding\stanford-corenlp-full-2017-06-09
#command line: java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

#ref.:https://stanfordnlp.github.io/CoreNLP/corenlp-server.html
#ref.:https://stanfordnlp.github.io/CoreNLP/corenlp-server.html#starting-the-server
#ref.:https://github.com/smilli/py-corenlp


## Document similarity
Calculating document similarity is the first step in many text analytic tasks, including clustering and finding similar documents to a single one known to be of relevance. In this part of the assignment, you’ll compare different ways of computing document similarity.


In [6]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

### 1.) Splitting, tokenization, normalization. 
In order to have a set of documents to calculate similarities between, split the classbios.txt file into one file per person. (Note that you can use regular expressions to do this.) Then, tokenize and normalize these files using any of the methods (in Lucene or CoreNLP) you used on the previous assignment. Next, remove stopwords and any words that contain non-alphabetic characters (e.g., puncutation or numbers), using any method you like. [No need to say anything about this problem in your write-up, but do include the code in your zip.]

In [7]:
text_file=open('classbios.txt', 'r',encoding="utf8")
text_file=text_file.read()
#print(text_file)

In [8]:
#tokenize and normalize
text_split = nlp.annotate(text_file, properties={
  'annotators': 'tokenize, lemma',
  'outputFormat': 'json'
  })

In [9]:
#text_split

In [10]:
#join split lemmas into list_of_tokens:
tokens=[]
for s in text_split['sentences']:
    tokens_from_this_s=[lemma['lemma'] for lemma in s['tokens']]
    tokens.append(tokens_from_this_s)

#tokens

In [11]:
#remove stop words using nltk
from nltk.corpus import stopwords

reduced_tokens=[]
stopset = ['I','a','and','to','the','in','of','my','for','with','that','as','at','from','is','on','have','me','be','an','it','this',
          ',', '.', '------------------------------------------------------------', ':', '**']
cachedStopWords = stopset+stopwords.words("english")

for each in tokens:
    each=[token for token in each if token not in cachedStopWords]
    reduced_tokens.append(each)

#reduced_tokens

In [12]:
#join the list of sentence'tokens into one big list of all tokens:
flat_reduced_tokens=[]
for token_sentence in reduced_tokens:
    flat_reduced_tokens=flat_reduced_tokens+[token for token in token_sentence]

#flat_reduced_tokens

In [13]:
# split list of all tokens into .txt file docs by 'hw0' for each classmate
tokens_one_doc=[]
list_docs=[]
flat_reduced_tokens.append('hw0')
for token in flat_reduced_tokens:
    if token != 'hw0':
        tokens_one_doc.append(token) #keep appending tokens from one doc to the same doc
    else:
        if len(tokens_one_doc)>0:
            list_docs.append(tokens_one_doc) #append completed doc to list of docs
            tokens_one_doc=[] #reset doc

#list_docs

In [14]:
#write list of docs out to local txt files:
import os

for doc in list_docs: 
    if len(doc)!=0:
        doc_text= " ".join([token for token in doc])
        with open(os.path.join('C:\\Users\Dingding\class_bios_docs',(doc[0]+' '+doc[1]).lstrip()+'.txt'), "w",encoding="utf-8") as file: 
            file.write(doc_text)


### 2.) Boolean similarity. 

Use the normalized, tokenized bio documents to create a binary term-document matrix, where each row represents a vocabulary item, each column represents a document, and each element is a 0 or 1. In this matrix, each document is represented by a binary vector. From this binary matrix, create a document similarity matrix, where cell i, j gives the similarity of documents i and j, and where similarity is given by the cosine between the document vectors: I⋅J|I||J|I⋅J|I||J|. Save this matrix to a file boolean.txt, where values are separated by a space, and where the rows and columns are arranged in the same order they were given in classbios.txt. Do not include any row or column labels in this file. In your write-up, answer: Which three pairs of documents are the most similar? Which three pairs are the least similar?

In [15]:
import numpy as np

In [16]:
first_token = list_docs[0][0]

In [17]:
n_Token = 0
token_dict = {first_token:n_Token}
n_Token += 1

In [18]:
n_Doc = len(list_docs)
boolMatrix = np.zeros([1,n_Doc])
boolMatrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [19]:
# edit boolean matrix
idx_Doc = 0
for doc in list_docs:
    for token in doc:
        
        if token not in token_dict.keys(): # add a new token
            token_dict[token] = n_Token
            boolMatrix = np.concatenate([boolMatrix,np.zeros([1,n_Doc])])
            n_Token += 1
        # register token into boolMatrix
        idx_Token = token_dict[token]
        boolMatrix[idx_Token,idx_Doc] = 1
            
    idx_Doc += 1 # go to next bio

boolMatrix

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  1.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [20]:
# BOOLEAN SIMILARITY
boolSimMatrix = np.zeros([n_Doc,n_Doc])
boolSimMatrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [21]:
top3Value = np.array([-1,-1,-1.0]); 
top3Idx = [-1,-1,-1]; # ordered
bottom3Value = np.array([2,2,2.0]); 
bottom3Idx = [-1,-1,-1]; 

In [22]:
for i in range(n_Doc):
    for j in range(n_Doc):
        
        if j == i:
            continue
        if j < i: # similarity has been calculated
            boolSimMatrix[i,j] = boolSimMatrix[j,i]
        else:
            
            Vec_i = boolMatrix[:,i]; Vec_j = boolMatrix[:,j]
            
            Sim_ij = np.sum(Vec_i*Vec_j)/(np.sum(Vec_i)*np.sum(Vec_j))
            boolSimMatrix[i,j] = Sim_ij
boolSimMatrix

array([[ 0.        ,  0.00117782,  0.00099433, ...,  0.00142954,
         0.00070458,  0.00077039],
       [ 0.00117782,  0.        ,  0.00097001, ...,  0.00138351,
         0.00091646,  0.00089928],
       [ 0.00099433,  0.00097001,  0.        , ...,  0.0011236 ,
         0.00093037,  0.00090289],
       ..., 
       [ 0.00142954,  0.00138351,  0.0011236 , ...,  0.        ,
         0.0011269 ,  0.00106456],
       [ 0.00070458,  0.00091646,  0.00093037, ...,  0.0011269 ,
         0.        ,  0.00085305],
       [ 0.00077039,  0.00089928,  0.00090289, ...,  0.00106456,
         0.00085305,  0.        ]])

In [23]:
# write matrix into a txt file
np.savetxt('boolean.txt',boolSimMatrix,fmt='%.5f')

#### top three similar pair of bios in cosine distance

In [123]:
#get top three and bottom three values in boolean similarity matrix:
#get top three values:
idx_array_top3value=boolSimMatrix.max(0).argsort()[-4:][::-1] #top three value index in array of top value in each document
top3value=boolSimMatrix.max(0)[idx_array_top3value].tolist()
#mark top three values in boolean similarity matrix:
top3_boolMatrix=np.isin(boolSimMatrix, top3value)
top3_boolMatrix

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [124]:
#get index of top three values:
top3_index=np.where(top3_boolMatrix) #first array is row index, second array is column index
top3_index

(array([ 7, 20, 22, 33], dtype=int64), array([33, 22, 20,  7], dtype=int64))

In [125]:
#retrieve top 3 pairs of most similar bios according to cosine distance: returned four pair of indexes (tie situation)
for x in range(0,4):
    #print(top3value[x]) #cosine similarity score
    print(" ".join(list_docs[top3_index[0][x]]))
    print(" ")
    print(" ".join(list_docs[top3_index[1][x]]))
    print(" ")

MEGHAN MENGHAN DING originally Shenzhen southern part China come United State senior year high school Rochester NY 2009 complete undergraduate study Northwestern 2014 degree statistics Economics go work investment management project management 2 year decide take technical path business world come back Northwestern msia program become interested text analytic since join msia program hear term learn see people present work text analytic either class industry amaze amount value extract unstructured datum want learn powerful tool future plan career datum science especially decision science side believe text analytic natural language processing one major distinguishing point make better datum scientist another major reason decide text analytic class pick python MSiA fall love elegant language want become better
 
CHAORAN WEI name Chaoran Wei MSiA program China undergraduate research project machine learning interested idea solve real world problem datum choose go MSiA get master degree want

#### bottom three similar pair of bios in cosine distance

In [91]:
#get bottom three and bottom three values in boolean similarity matrix:
#get bottom three values:
idx_array_bottom3value=boolSimMatrix.min(0).argsort()[-3:] #bottom three value index in array of top value in each document
bottom3value=boolSimMatrix.max(0)[idx_array_bottom3value].tolist()
#mark bottom three values in boolean similarity matrix:
bottom3_boolMatrix=np.isin(boolSimMatrix, bottom3value)
bottom3_boolMatrix

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [92]:
#get index of bottom three values:
bottom3_index=np.where(bottom3_boolMatrix) #first array is row index, second array is column index
bottom3_index

(array([ 7,  7, 15, 23, 34, 35], dtype=int64),
 array([15, 35,  7, 34, 23,  7], dtype=int64))

In [93]:
#retrieve  bottom 3 pairs of most similar bios according to cosine distance: returned four pair of indexes (tie situation)
for x in range(0,4):
    #print(bottom3value[x]) #cosine similarity score
    print(" ".join(list_docs[bottom3_index[0][x]]))
    print(" ")
    print(" ".join(list_docs[bottom3_index[1][x]]))
    print(" ")

MEGHAN MENGHAN DING originally Shenzhen southern part China come United State senior year high school Rochester NY 2009 complete undergraduate study Northwestern 2014 degree statistics Economics go work investment management project management 2 year decide take technical path business world come back Northwestern msia program become interested text analytic since join msia program hear term learn see people present work text analytic either class industry amaze amount value extract unstructured datum want learn powerful tool future plan career datum science especially decision science side believe text analytic natural language processing one major distinguishing point make better datum scientist another major reason decide text analytic class pick python MSiA fall love elegant language want become better
 
SIK JUNG Seoul Korea grow since 15 move Shanghai China attend high school move Los Angeles CA attend UCLA Evanston il come msia want datum science field first hear field take stati

## tf-idf similarity

Use the normalized, tokenized bio documents to create a term-document matrix of raw frequencies. Then transform this raw frequency matrix into a tf-idf matrix by using the formula (1+log(tf))(log(Ndf))(1+log⁡(tf))(log⁡(Ndf)) where tftf is the term’s raw frequency in this document, dfdf is the number of documents the term occurs in, and NN is the total number of documents. Now, again, convert this term-document matrix into a document similarity matrix in the same way as before and save it as tf_idf.txt, in the same format as before. In your write-up, answer: Which three pairs of documents are the most similar now? Which three pairs are the least similar? Does this more advanced method seem to be doing a better job than just using binary similarity? Why?

In [105]:
n_Token
n_Doc

36

In [106]:
freqMatrix = np.zeros([n_Token,n_Doc])
freqMatrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [108]:
# edit boolean matrix
idx_Doc = 0
for doc in list_docs:
    for token in doc:
        # register token into boolMatrix
        idx_Token = token_dict[token] 
        freqMatrix[idx_Token,idx_Doc] += 1
            
    idx_Doc += 1 # go to next bio
freqMatrix        

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 2.,  2.,  0., ...,  0.,  4.,  4.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  2.],
       [ 0.,  0.,  0., ...,  0.,  0.,  2.],
       [ 0.,  0.,  0., ...,  0.,  0.,  2.]])

In [109]:
# frequency matrix to tf-idf matrix
tfidfMatrix = np.zeros([n_Token,n_Doc])

In [111]:
for idx_Token in range(n_Token):
    for idx_Doc in range(n_Doc):
        
        tf = freqMatrix[idx_Token,idx_Doc]   
        
        if tf == 0:
            continue
        df = (freqMatrix[idx_Token,:]>0.1).sum()
        tfidfMatrix[idx_Token,idx_Doc] = (1 + np.log(tf)) * np.log(n_Doc/float(df))
tfidfMatrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 8.55133104,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.37302421,  1.37302421,  0.        , ...,  0.        ,
         1.9351182 ,  1.9351182 ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  6.06742499],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  6.06742499],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  6.06742499]])

In [112]:
# initiate tfidf similarity matrix
tfidfSimMatrix = np.zeros([n_Doc,n_Doc])

In [114]:
for i in range(n_Doc):
    for j in range(n_Doc):        
        if j == i:
            continue
        if j < i: # similarity has been calculated
            tfidfSimMatrix[i,j] = tfidfSimMatrix[j,i]
        else:
            Vec_i = tfidfMatrix[:,i]; Vec_j = tfidfMatrix[:,j]
            sim_ij = np.sum(Vec_i*Vec_j)/(np.sum(Vec_i)*np.sum(Vec_j))
            tfidfSimMatrix[i,j] = sim_ij
tfidfSimMatrix

array([[ 0.        ,  0.00037603,  0.00037602, ...,  0.0004653 ,
         0.0001906 ,  0.00015558],
       [ 0.00037603,  0.        ,  0.00044632, ...,  0.00044872,
         0.00029046,  0.00022938],
       [ 0.00037602,  0.00044632,  0.        , ...,  0.00036112,
         0.00032444,  0.00027729],
       ..., 
       [ 0.0004653 ,  0.00044872,  0.00036112, ...,  0.        ,
         0.00051106,  0.00031779],
       [ 0.0001906 ,  0.00029046,  0.00032444, ...,  0.00051106,
         0.        ,  0.0003558 ],
       [ 0.00015558,  0.00022938,  0.00027729, ...,  0.00031779,
         0.0003558 ,  0.        ]])

In [115]:
# write ifidf similarity matrix into a txt file
np.savetxt('tf_idf.txt',tfidfSimMatrix,fmt='%.5f')

#### bottom three similar pair of bios in cosine distance for tf-idf matrix

In [116]:
#get bottom three and bottom three values in tfidf similarity matrix:
#get bottom three values:
idx_array_bottom3value=tfidfSimMatrix.min(0).argsort()[-3:] #bottom three value index in array of top value in each document
bottom3value=tfidfSimMatrix.max(0)[idx_array_bottom3value].tolist()
#mark bottom three values in boolean similarity matrix:
bottom3_tfidfMatrix=np.isin(tfidfSimMatrix, bottom3value)
bottom3_tfidfMatrix

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [117]:
#get index of bottom three values:
bottom3_index=np.where(bottom3_tfidfMatrix) #first array is row index, second array is column index
bottom3_index

(array([ 7, 15, 28, 33, 34, 35], dtype=int64),
 array([35, 28, 15, 34, 33,  7], dtype=int64))

In [118]:
#retrieve  bottom 3 pairs of most similar bios according to cosine distance: returned 3 pair of indexes (tie situation)
for x in range(0,4):
    #print(bottom3value[x]) #cosine similarity score
    print(" ".join(list_docs[bottom3_index[0][x]]))
    print(" ")
    print(" ".join(list_docs[bottom3_index[1][x]]))
    print(" ")

MEGHAN MENGHAN DING originally Shenzhen southern part China come United State senior year high school Rochester NY 2009 complete undergraduate study Northwestern 2014 degree statistics Economics go work investment management project management 2 year decide take technical path business world come back Northwestern msia program become interested text analytic since join msia program hear term learn see people present work text analytic either class industry amaze amount value extract unstructured datum want learn powerful tool future plan career datum science especially decision science side believe text analytic natural language processing one major distinguishing point make better datum scientist another major reason decide text analytic class pick python MSiA fall love elegant language want become better
 
KEVIN ZHAI year ago dad study English college Nanjing China one day history professor US come college give lecture dad choose translator lecture professor dad talk one reason anoth

#### top three similar pair of bios in cosine distance for tf-idf matrix

In [126]:
#get top three and bottom three values in tfidf similarity matrix:
#get top three values:
idx_array_top3value=tfidfSimMatrix.max(0).argsort()[-5:][::-1] #top three value index in array of top value in each document
top3value=tfidfSimMatrix.max(0)[idx_array_top3value].tolist()
#mark top three values in boolean similarity matrix:
top3_tfidfMatrix=np.isin(tfidfSimMatrix, top3value)
top3_tfidfMatrix

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [127]:
#get index of top three values:
top3_index=np.where(top3_tfidfMatrix) #first array is row index, second array is column index
top3_index

(array([ 7,  8, 20, 22, 22, 28], dtype=int64),
 array([28, 22, 22,  8, 20,  7], dtype=int64))

In [128]:
#retrieve top 3 pairs of most similar bios according to cosine distance: returned four pair of indexes (tie situation)
for x in range(0,4):
    #print(top3value[x]) #cosine similarity score
    print(" ".join(list_docs[top3_index[0][x]]))
    print(" ")
    print(" ".join(list_docs[top3_index[1][x]]))
    print(" ")

MEGHAN MENGHAN DING originally Shenzhen southern part China come United State senior year high school Rochester NY 2009 complete undergraduate study Northwestern 2014 degree statistics Economics go work investment management project management 2 year decide take technical path business world come back Northwestern msia program become interested text analytic since join msia program hear term learn see people present work text analytic either class industry amaze amount value extract unstructured datum want learn powerful tool future plan career datum science especially decision science side believe text analytic natural language processing one major distinguishing point make better datum scientist another major reason decide text analytic class pick python MSiA fall love elegant language want become better
 
ERIC PAWLAKOS bear raise Bay Area grow Walnut Creek move around 8th grade bring msia interest statistics undergraduate major Math/Econ find interesting class statistics econometric